# 【第3週】<B>ベースラインを作って提出してみよう！！</B>

- ベースライン構築
- 特徴量エンジニアリング

|#|分析プロセス|タスク|①ベースライン作成|②特徴量エンジニアリング|③モデルチューニング|
|:---:|:---:|:---:|:---:|:---:|:---:|
|1-1|分析設計|問題の理解|⚪︎|||
|1-2||分析の設計|⚪︎|||
|2-1|データ前処理|ファイルの読み込み|⚪︎|||
|2-2||データの確認(簡易)|⚪︎|||
|2-3||データの確認(詳細)||⚪︎||
|2-4||欠損値の対応||⚪︎||
|2-5||外れ値の対応||⚪︎||
|3-1|特徴量生成|単変数:数値||⚪︎||
|3-2||単変数:カテゴリ変数||⚪︎||
|3-3||2変数組合せ||⚪︎||
|3-4||テキストデータ||⚪︎||
|4-1|データセット作成|特徴量選択||⚪︎||
|4-2||データセット作成|⚪︎|||
|5-1|バリデーション設計|バリデーション方法|⚪︎|||
|5-2||評価指標|⚪︎|||
|6-1|モデル学習|勾配ブースティング|⚪︎|||
|6-2||ハイパーパラメータチューニング|||⚪︎|
|6-3||scikit-learnのモデル|||⚪︎|
|6-4||ニューラルネットワーク|||⚪︎|
|6-5||アンサンブル|||⚪︎|
|7-1|モデル推論|推論用データセット作成|⚪︎|||
|7-2||学習済モデルを用いた推論|⚪︎|||

# 先週までの振り返り

1. polarsで遊んだ
2. titanicデータセットを見てみた
3. ベースラインを作る途中←イマココ

In [ ]:
# polarsの読み込みとデータの読み込み
import polars as pl
train = pl.read_csv("../input/titanic/train.csv")

In [ ]:
# データの上位5件を確認する
train.head()

In [ ]:
# 性別は重要なデータ。学習に用いたいが文字なため、数字に変換する必要あり。女性が0で男性が1にします。
train = train.with_columns(
    (pl.col("Sex") == "male").cast(pl.Int8).alias("Sex")
)

In [ ]:
# 必要な列を選択し、欠損値を処理
train = train.select([
   "Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"
]).drop_nulls()

In [ ]:
# x = train.with_columns(pl.exclude("Survived")).to_numpy()  # 説明変数
x = train.select(pl.exclude("Survived")).to_numpy()  # 説明変数
y = train.select(["Survived"]).to_numpy()  # 目的変数

### 機械学習の基本的な考え方
<B>x(予測の根拠となる数字: 説明変数) → AI(推論) → y(予測したい数字: 目的変数)</B>

説明変数と目的変数 = 特徴量(features)


In [ ]:
from sklearn.model_selection import train_test_split
# データを訓練用とテスト用に分割
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

|X_train|X_val|y_train|y_val|
|---|---|---|---|
|AIに学ばせる用の説明変数|学んだAIを評価するための説明変数|AIに学ばせる用の目的変数|学んだAIを評価するための説明変数|
|8|2|8|2|

In [ ]:
# LightGBMを用意
#データセットを作成
import lightgbm as lgb
trains = lgb.Dataset(X_train, y_train)
valids = lgb.Dataset(X_val, y_val)

In [ ]:
# LightGBMにどんなタスクを振るかを指示
params = {
    "objective": 'binary',  # "multiclass", "regression"
    "metrics": "acc"  # "測度", "評価指標", acc...accuracy(正解率)
}

In [ ]:
model = lgb.train(
    params,  # さっき設定したやつ
    trains,  # 訓練用データセット
    valid_sets=valids,  # 評価用データセット
    num_boost_round=50  # どれくらい弱学習器を作るか
)

## テストデータによる推論

In [ ]:
# polarsの読み込みとデータの読み込み
import polars as pl
test = pl.read_csv("../input/titanic/test.csv")
test_id = test.select("PassengerId")

In [ ]:
# 性別は重要なデータ。学習に用いたいが文字なため、数字に変換する必要あり。女性が0で男性が1にします。
test = test.with_columns(
    (pl.col("Sex") == "male").cast(pl.Int8).alias("Sex")
)

In [ ]:
# 必要な列を選択し、欠損値を処理
test = test.select([
    "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"
])

In [ ]:
x_test = test.to_numpy()  # 説明変数

In [ ]:
# TypeError: Cannot use Dataset instance for prediction, please use raw data instead
# tests = lgb.Dataset(x_test)

In [ ]:
# y_pred = model.predict(tests)
y_pred = model.predict(x_test)

In [ ]:
y_pred

In [ ]:
y_pred.round()

In [ ]:
sample_sub = pl.read_csv("../input/titanic/gender_submission.csv")

In [ ]:
sample_sub

In [ ]:
sub = pl.concat([test_id, pl.DataFrame(y_pred.round(0).astype(int)).rename({"column_0": "Survived"})], how="horizontal")

In [ ]:
sub

In [ ]:
sub.write_csv("submission.csv")